<a href="https://colab.research.google.com/github/cathieG/Prompt_Engineering/blob/main/few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run the dependencies

!pip3 install openai
!pip install evaluate
!pip install sacrebleu
! git clone -q https://github.com/microsoft/CodeXGLUE.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and pl

In [ ]:
"""Run this model in Python

> pip install openai
"""
import os
import pandas as pd
from openai import OpenAI
import evaluate
import json


sacrebleu = evaluate.load("sacrebleu")

# Create your PAT token by following instructions here:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# grab the token from the token.txt file
# with open("token.txt", "r") as f:
#     GITHUB_TOKEN = f.read().strip()

# Set it as an environment variable
# Insert your github token next to this line. As my github token is private, I obviously will not be showcasing it here.
# os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN


# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

# List of available models
model_choices = ["gpt-4o-mini" , "Codestral-2501" , "DeepSeek-R1"]
#DeepSeek is not allowed on gov owened machines

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Workflow:**

* The following cells define the few-shot prompts for the 22 tasks.

* They will be ran using a loop on two models after the definitions.

In [ ]:
# Task 1:
# Code Summarization (Java) Summarize the functionality of the following method:

code1 = '''
    Prompt: Please provide a two-sentence or less summary of the following method:


    public Map < String , Integer > countWordFrequency ( List < String > words ) {
    Map < String , Integer > freqMap = new HashMap < >() ;
    for ( String word : words ) {
        freqMap . put ( word , freqMap . getOrDefault ( word , 0) + 1) ;
    }
    return freqMap ;
    }

'''
few_shot_prompt1 = '''
    You are a helpful assistant that summarizes the functionality of Java codes.

    ### Example 1
    Code:
    public int add(int a, int b) {
        return a + b;
    }
    Summary:
    Returns the sum of two integers.

    ### Example 2
    Code:
    public boolean isEven(int number) {
        return number % 2 == 0;
    }
    Summary:
    Checks whether a number is even.
    ### Now summarize the following method:


    Code:
    ''' + code1 + '''
    Summary:
    '''
# response = client.chat.completions.create(
#     model = model_choices[1],  # Choose the model you want to use
#     messages = [
#         {"role": "user", "content": few_shot_prompt1},
#     ],
#     max_tokens = 1024,
#     temperature = 0.7,
# )

# print(response.choices[0].message.content)

In [ ]:
# Task 2
# Fix one-off buggy Python code

code2 = '''
    def sum_range ( start , end ) :
        total = 0
        for i in range ( start , end ) :
            total += i
        return total

'''

few_shot_prompt2 = '''
    Your job is to identify the one-off bugs in python codes and fix them.

    ### Example 1
    The function calculates the sum of all integers from start to end, inclusive. However, the original function uses range(start, end), which excludes the end value. This leads to an incomplete summation by omitting the final number.
    Original Code:
    def sum_range(start, end):
        total = 0
       for i in range(start, end):
           total += i
       return total

    Corrected Code:
    def sum_range(start, end):
        total = 0
       for i in range(start, end + 1):
            total += i
       return total

    ### Example 2
    The function is intended to add up all the integers from start to end, including both endpoints. But because range(start, end) doesn't include end, the function misses the last number and gives a result that's too small.
    Original Code:
    def sum_range(start, end):
        total = 0
        for i in range(start, end):
            total += i
            return total

    Corrected Code:
    def sum_range(start, end):
        total = 0
        for i in range(start, end + 1):
            total += i
        return total

Now, given the code: ''' + code2 + '''
    Describe the function, identify the one-off bug,
    and correct the code like the examples. You only need to
    return the corrected code.

'''


In [ ]:
# Task 3
# Bug Classification in C++ functions

code3 = '''
    int * getArray (int size ) {
        int arr [ size ]; // Warning : local array
        return arr ; // Bug: returning pointer to local variable
}

'''

few_shot_prompt3 = '''
    Your job is to classify the type of bug C++ functions.
    ### Example 1:
    code:
    int* getArray(int size) {
    int arr[size]; // Warning: local array
    return arr;    // Bug: returning pointer to local variable}

    Bug Classification: Returning pointer to local variable
    Detailed Bug Explanation: The function attempts to return an array of integers.
    However, the array is declared as a local variable inside the function.
    Returning a pointer to this local array is invalid because the memory is deallocated
    once the function exits, leading to undefined behavior.

    ### Example 2:
    code:
    void printArray(int* arr, int size) {
    for (int i = 0; i <= size; ++i) {
        std::cout << arr[i] << std::endl; } }

    Bug Classification: Off-by-one error (out-of-bounds access)
    Detailed Bug Explanation: This function uses a loop to iterate through an array,
    but it goes one step beyond the last valid index. Since valid indices go from 0 to
    size - 1, accessing index size causes undefined behavior.

    Now, given the code ''' + code3 + '''
    Classify the type of bug in the c++ function. You only need to
    return the bug classification.

'''



In [ ]:
# Task 4
# Generating Email Validators (Python + Regex)

code4 = '''

    def is_valid_email ( email ):
    # TODO : Complete using regex
    pass
'''

few_shot_prompt4 = '''

    You are to help complete a function using regex to validate email addresses.
    ### Example 1:

    def is_valid_email(email):
        import re
        pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        return re.match(pattern, email) is not None


    ### Example 2:

    def is_valid_email(email):
        import re
        pattern = r'^[a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return re.match(pattern, email) is not None


    Now, complete the following function using regex to validate
    basic email addresses. You just need to provide the final function.
    No need for explanations or examples.: ''' + code4



In [ ]:
# Task 5
# Generating Flask APIs (Python)

code5 = '''
    from flask import Flask , jsonify
    app = Flask ( __name__ )
    @app . route (’/ greet / < username >’)
    def greet ( username ) :
        # TODO : Return a JSON greeting
        pass

'''

few_shot_prompt5 = '''
    You job is to create a ‘/greet/<username>‘ endpoint that returns a JSON greeting:

    ### Example 1:
    Original code:
    from flask import Flask, jsonify
    app = Flask(__name__)

    @app.route('/greet/<username>')
    def greet(username):
        # TODO: Return a JSON greeting
        pass

    Completed code:
    from flask import Flask, jsonify
    app = Flask(__name__)

    @app.route('/greet/<username>')
    def greet(username):
        return jsonify({'message': f'Hello, {username}!'})


    ### Example 2:
    Original code:
    from flask import Flask, jsonify
    app = Flask(__name__)

    @app.route('/greet/<username>')
    def greet(username):
        # TODO: Return a JSON greeting
        pass

    Completed code:
    from flask import Flask, jsonify
    app = Flask(__name__)

    @app.route('/greet/<username>')
    def greet(username):
        response = {'message': 'Hello, ' + username + '!'}
        return jsonify(response)

    Now, create a ‘/greet/<username>‘
    endpoint that returns a JSON greeting given the code.
    You only need to provide the corrected code. No need for
    explanations or examples: ''' + code5




In [ ]:
# Task 6
# SQL Schema Design (SQL)

code6 = '''
        -- TODO : Design schema with appropriate keys and constraints
        -- Tables : users (id , name ), books (id , title ), reviews (id , user_id , book_id , rating )
    '''

few_shot_prompt6 = '''
    Your job is to write the schema for a review app with users, books, and reviews.

    ### Example 1:
    Given:
        -- TODO : Design schema with appropriate keys and constraints
        -- Tables: customers (id, name), products (id, name), orders (id, customer_id, product_id, quantity)

    Completed task:
    CREATE TABLE customers (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    );

    CREATE TABLE products (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    );

    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        FOREIGN KEY (customer_id) REFERENCES customers(id),
        FOREIGN KEY (product_id) REFERENCES products(id)
    );


    ### Example 2:
    Given:
        -- TODO : Design schema with appropriate keys and constraints
        -- Tables: users (user_id, username), movies (movie_id, title), ratings (rating_id, user_id, movie_id, score)

    Completed task:
    CREATE TABLE users (
        user_id INTEGER PRIMARY KEY,
        username TEXT NOT NULL
    );

    CREATE TABLE movies (
        movie_id INTEGER PRIMARY KEY,
        title TEXT NOT NULL
    );

    CREATE TABLE ratings (
        rating_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        movie_id INTEGER,
        score INTEGER CHECK(score BETWEEN 1 AND 5),
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
    );


    Now, do the same for the following code.
    No need for explanations or examples: ''' + code6



In [ ]:
# Task 7
# Null Dereference Detection (Java)

code7 = '''
    public int getLength ( String s ) {
    return s . length () ; // What if s is null ?
    }

'''

few_shot_prompt7 = '''
    Your job is to identify potential null dereference bugs in Java code. These bugs happen when a method is called on an object that might be null, which can lead to a NullPointerException at runtime.

    ### Example 1:
    Original code:
    public String getUpperCase(String input) {
        return input.toUpperCase(); // Potential null dereference
    }

    Issue:
    The null dereference may occur on **line 2**, where `input.toUpperCase()` is called without checking if `input` is null.

    ### Example 2:
    Original code:
    public int getNameLength(User user) {
        return user.getName().length(); // Potential null dereference
    }

    Issue:
    A null dereference may occur on **line 2**, where `user.getName()`
    could be null, and `.length()` is called on it without validation.

    Now, identify any potential null dereference issue in the following code.
    You only need to provide the issues. No need for explanations
    or examples:''' + code7



In [ ]:
# Task 8
# CSV Parser Variants (Python)

code8 = '''
    def parse_csv_line ( line ) :
    return line . split (’,’) # Incomplete : doesn ’t handle quoted fields

'''

few_shot_prompt8 = '''
    Your job is to identify limitations in naive CSV line parsers and suggest improvements. Naively splitting on commas does not handle quoted fields correctly. Provide examples of this issue.

    ### Example 1:
    Original code:
    def parse_csv_line(line):
        return line.split(',')

    Issue:
    This approach fails for input like `"Smith, John",25,"New York, NY"` because it will incorrectly split inside quoted strings. Fields containing commas must be treated as a single value if enclosed in quotes.

    ### Example 2:
    Original code:
    def parse_csv_line(line):
        return [cell.strip() for cell in line.split(',')]

    Issue:
    Even though this adds whitespace trimming, it still cannot
    handle lines like: `"Doe, Jane", "Engineer, Senior", "R&D"`,
    where some fields contain commas. It will split those fields incorrectly.

    Now, improve the code. You only need to provide me with
    the improved code. No need for explanations or examples:''' + code8

In [ ]:
# Task 9
# Data Class to API Conversion (Kotlin) Convert the data class to a REST API using Ktor:

code9 = '''
    data class Product ( val id : Int , val name : String , val price : Double )
    // TODO : Create GET and POST endpoints using Ktor

'''

few_shot_prompt9 = '''
    Your task is to convert a Kotlin data class into a simple REST API using Ktor. For each example, create GET and POST routes that handle objects of the data class, using in-memory storage.

    ### Example 1:
    Original code:
    data class User(val id: Int, val name: String)
    // TODO: Create GET and POST endpoints using Ktor

    Completed code:
    import io.ktor.application.*
    import io.ktor.response.*
    import io.ktor.request.*
    import io.ktor.routing.*
    import io.ktor.features.*
    import io.ktor.serialization.*
    import io.ktor.server.engine.*
    import io.ktor.server.netty.*

    val users = mutableListOf<User>()

    fun main() {
        embeddedServer(Netty, port = 8080) {
            install(ContentNegotiation) {
                json()
            }
            routing {
                get("/users") {
                    call.respond(users)
                }
                post("/users") {
                    val user = call.receive<User>()
                    users.add(user)
                    call.respond(user)
                }
            }
        }.start(wait = true)
    }

    ### Example 2:
    Original code:
    data class Book(val id: Int, val title: String, val author: String)
    // TODO: Create GET and POST endpoints using Ktor

    Completed code:
    import io.ktor.application.*
    import io.ktor.response.*
    import io.ktor.request.*
    import io.ktor.routing.*
    import io.ktor.features.*
    import io.ktor.serialization.*
    import io.ktor.server.engine.*
    import io.ktor.server.netty.*

    val books = mutableListOf<Book>()

    fun main() {
        embeddedServer(Netty, port = 8080) {
            install(ContentNegotiation) {
                json()
            }
            routing {
                get("/books") {
                    call.respond(books)
                }
                post("/books") {
                    val book = call.receive<Book>()
                    books.add(book)
                    call.respond(book)
                }
            }
        }.start(wait = true)
    }

    Now, convert the following data class into a REST API using Ktor.
    You only need to provide the final code. No need for explanations
    or examples:''' + code9


In [ ]:
# Task 10
# Function Summarization (Python) Write a brief summary of the function:

code10 = '''
    def reverse_words ( sentence ) :
    return ’ ’. join ( sentence . split () [:: -1])

'''

few_shot_prompt10 = '''
    You are given a Python function. Write a brief summary of what the function does.

    ### Example 1:
    Function:
    def square_elements(nums):
        return [x ** 2 for x in nums]

    Summary:
    This function takes a list of numbers and returns a new list containing the squares of each number.

    ### Example 2:
    Function:
    def is_palindrome(s):
        return s == s[::-1]

    Summary:
    This function checks whether a string is a palindrome by comparing it to its reverse.

    ### Example 3:
    Function:
    def count_vowels(s):
        return sum(1 for c in s.lower() if c in 'aeiou')

    Summary:
    This function counts the number of vowels in a given string.

    Now, summarize the following function in two sentences or less:

    Function:
    ''' + code10

In [ ]:
# Task 11
# Prompt from Code Comments (Python) Write a prompt that could generate the code:

code11 = '''
    # This function checks if a number is prime
    def is_prime ( n ) :
    if n <= 1:
    return False
    for i in range (2 , int( n **0.5) +1) :
    if n % i == 0:
    return False
    return True

'''

few_shot_prompt11 = '''
    You are given a Python function with comments. Write a natural language prompt that could generate this code.

    ### Example 1:
    Code:
    # This function returns the factorial of a number
    def factorial(n):
        if n == 0:
            return 1
        return n * factorial(n - 1)

    Prompt:
    Write a recursive Python function to compute the factorial of a number.

    ### Example 2:
    Code:
    # This function returns the nth Fibonacci number using recursion
    def fibonacci(n):
        if n <= 1:
            return n
        return fibonacci(n - 1) + fibonacci(n - 2)

    Prompt:
    Create a Python function that returns the nth Fibonacci number using recursion.

   Now, write a prompt that would result in the generation
   of the following code. No need to
   explain your reasoning or give examples: ''' + code11



In [ ]:
# Task 12
# Fixing Factorial Bug (Python) Fix the bug when input is 0:

code12 = '''
    def factorial ( n ) :
        result = 1
        for i in range (1 , n ) :
          result *= i
        return result



'''

few_shot_prompt12 = '''
    You are given a Python function that tries to compute the factorial of a number. If there’s a bug when the input is 0, identify the issue and correct the function.

    ### Example 1:

    Original Code:
    def factorial(n):
        result = 1
        for i in range(1, n):
            result *= i
        return result

    Bug:
    This function fails when n = 0. The factorial of 0 should be 1, but this code returns 1 without any iteration, which may seem fine, but it also gives incorrect results for n = 1 because it doesn’t include n itself in the loop.

    Corrected Code:
    def factorial(n):
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

    ### Example 2:

    Original Code:
    def factorial(n):
        if n == 0:
            return 0
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

    Bug:
    The function incorrectly returns 0 for n = 0. Mathematically, 0! is 1.

    Corrected Code:
    def factorial(n):
        if n == 0:
            return 1
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

    ### Now, fix the bug in the following code if the input is 0.
    you only need to provide me with the corrected code. No need
    for explanation or examples:
    ''' + code12

In [ ]:
# Task 13
# Linked List Node Deletion (C) Implement node deletion by value:

code13 = '''
    struct Node {
      int data ;
      struct Node * next ;
    };
    void deleteNode ( struct Node ** head , int key ) {
      // TODO : Implement node deletion
    }

    '''

few_shot_prompt13 = '''
    You are given the definition of a singly linked list in C. Your task is to implement a function that deletes the first node with a given value from the list. Handle edge cases like deleting the head node or deleting a value that’s not in the list.

    ### Example 1:

    Given:
    struct Node {
      int data;
      struct Node* next;
    };

    void deleteNode(struct Node** head, int key) {
      // TODO: Implement node deletion
    }

    Completed:
    void deleteNode(struct Node** head, int key) {
      struct Node* temp = *head;
      struct Node* prev = NULL;

      // If head node itself holds the key
      if (temp != NULL && temp->data == key) {
        *head = temp->next;
        free(temp);
        return;
      }

      // Search for the key
      while (temp != NULL && temp->data != key) {
        prev = temp;
        temp = temp->next;
      }

      // If key was not found
      if (temp == NULL) return;

      // Unlink the node
      prev->next = temp->next;
      free(temp);
    }

    ### Example 2:

    Given:
    struct Node {
      int data;
      struct Node* next;
    };

    void deleteNode(struct Node** head, int key) {
      // TODO: Implement logic to remove node by value
    }

    Completed:
    void deleteNode(struct Node** head, int key) {
      if (*head == NULL) return;

      struct Node* curr = *head;
      struct Node* prev = NULL;

      while (curr != NULL) {
        if (curr->data == key) {
          if (prev == NULL) {
            // Deleting head node
            *head = curr->next;
          } else {
            prev->next = curr->next;
          }
          free(curr);
          return;
        }
        prev = curr;
        curr = curr->next;
      }
    }

    Now, implement the node deletion logic in the following code.
    You only need to provide the completed code.
    No need for explanations or examples:
    ''' + code13

In [ ]:
# Task 14
# Recursive Function Completion (Python) Complete the recursive function for Fibonacci:

code14 = '''
  def fibonacci ( n ) :
    # TODO
'''

few_shot_prompt14 = '''
    You are to complete recursive functions. Each function should include a base case and a recursive step.

    ### Example 1:

    def factorial(n):
        if n == 0 or n == 1:
            return 1
        return n * factorial(n - 1)

    This function returns the factorial of n using recursion. It stops when n is 0 or 1.

    ### Example 2:

    def sum_array(arr):
        if not arr:
            return 0
        return arr[0] + sum_array(arr[1:])
    This function returns the sum of all numbers in the list using recursion.
    It stops when the list is empty.

    Now, complete the recursive function to compute the nth Fibonacci number.
    you only need to provide the completed function. No need for explanations
    or examples:

    ''' + code14

In [ ]:
# Task 15
# Constructor Completion (Python) Complete the class constructor:

code15 = '''
  class Person :
    def __init__ ( self ) :
      # TODO : Add name , age , and optional email
      pass

'''

few_shot_prompt15 = '''
    You are to complete class constructors in Python by initializing attributes using `self`.

    ### Example 1:

    class Book:
        def __init__(self, title, author):
            self.title = title
            self.author = author
    This constructor initializes a `Book` object with a title and an author.


    ### Example 2:

    class Student:
        def __init__(self, name, grade, email=None):
            self.name = name
            self.grade = grade
            self.email = email

    This constructor initializes a `Student` with name and
    grade, and an optional email (defaulting to None).

    Now, complete the class constructor so it takes `name`,
    `age`, and optional `email`. No need for explanations or examples:

    ''' + code15

In [ ]:
# Task 16
# Binary Search Completion (Java) Complete the binary search implementation:

code16 = '''
    public int binarySearch ( int [] arr , int target ) {
      int left = 0 , right = arr . length - 1;
      while ( left <= right ) {
        int mid = ( left + right ) / 2;
        // TODO : Compare and adjust bounds
      }
      return -1;
    }

'''

few_shot_prompt16 = '''
    You are to complete the binary search implementation by comparing the mid value to the target and adjusting the bounds accordingly.

    ### Example 1:

    public int binarySearch(int[] arr, int target) {
        int left = 0, right = arr.length - 1;
        while (left <= right) {
            int mid = left + (right - left) / 2;
            if (arr[mid] == target) {
                return mid;
            } else if (arr[mid] < target) {
                left = mid + 1;
            } else {
                right = mid - 1;
            }
        }
        return -1;
    }
    This function implements a binary search to find the index of the target in a sorted array. It adjusts the left and right bounds based on comparisons with the middle value.

    ### Example 2:

    public int binarySearch(int[] arr, int target) {
        int left = 0, right = arr.length - 1;
        while (left <= right) {
            int mid = left + (right - left) / 2;
            if (arr[mid] == target) {
                return mid;
            } else if (arr[mid] < target) {
                left = mid + 1;
            } else {
                right = mid - 1;
            }
        }
        return -1;
    }

    This binary search function also looks for the target
    in a sorted array, adjusting the search bounds based on
    comparisons with the middle element.

    Now, complete the binary search implementation by
    filling in the missing steps. You only need to provide
    me with the completed constructor. No need for explanations or examples:

    ''' + code16

In [ ]:
# Task 17
# Self-Consistency Bug Fixing (C++) Resolve inconsistency between function name and logic:

code17 = '''
    // Supposed to return true if x is even
    bool isOdd (int x ) {
      return x % 2 == 0; // Logic contradicts function name
    }

'''

few_shot_prompt17 = '''
    Your task is to identify and resolve inconsistencies between function names and their logic.

    ### Example 1:

    Original code:

        // Supposed to return true if x is odd
        bool isEven(int x) {
            return x % 2 != 0; // Logic contradicts function name
        }

    Completed task:

        // Supposed to return true if x is odd
        bool isOdd(int x) {
            return x % 2 != 0; // Fixed: Corrected function name and logic
        }


    ### Example 2:

    Original code:

        // Supposed to return true if x is positive
        bool isNegative(int x) {
            return x > 0; // Logic contradicts function name
        }

    Completed task:

        // Supposed to return true if x is positive
        bool isPositive(int x) {
            return x > 0; // Fixed: Corrected function name and logic
        }

    Now, resolve the inconsistency in the following code.
    You only need to provide me the completed code. No need
    for explanation or examples:
    Code:
    ''' + code17

In [ ]:
# Task 18
# Prompt Chaining: Bug Identification → Fix (JavaScript) Identify and fix the bug:

code18 = '''
    function isEven ( n ) {
    return n % 2; // Returns 1 or 0 , not true / false
    }

'''

few_shot_prompt18 = '''
    Your task is to identify the bug in the JavaScript function and fix it.

    ### Example 1:

    Original code:

        function isEven(n) {
            return n % 2; // Returns 1 or 0, not true/false
        }

    Bug identification:
    The function returns `1` or `0`, but it should return `true` or `false` for even or odd numbers.

    Fix:

        function isEven(n) {
            return n % 2 === 0; // Corrected: Return true if even, false if odd
        }

    Explanation:
    The original code returned `1` or `0`, which is incorrect for an even/odd check. The fix uses a strict equality check (`=== 0`) to return `true` for even numbers and `false` for odd numbers.

    ### Example 2:

    Original code:

        function isPositive(n) {
            return n > 0 ? 1 : 'negative'; // Returns 1 or a string, not true/false
        }

    Bug identification:
    The function returns `1` for positive numbers and `'negative'` for non-positive numbers, but it should return `true` or `false`.

    Fix:

        function isPositive(n) {
            return n > 0; // Corrected: Return true if positive, false otherwise
        }

    Explanation:
    The function should return `true` for positive numbers and `false` for non-positive numbers. The original code returned `1` or a string, which doesn't match the intended behavior.

    Now, identify the bug and fix the following code:
    ''' + code18


In [ ]:
# Task 19
# Summary Decomposition (C++) Decompose the high-level comment/summary into logical steps:

code19 = '''
    // Function that validates an input , calculates square , and returns result
    int process (int x ) {
      if ( x < 0) return -1;
      return x * x ;
    }

'''

few_shot_prompt19 = '''
    Your task is to decompose the high-level comment into logical steps for the given code.

    ### Example 1:

    Original code:

        // Function that validates an input, calculates square, and returns result
        int process(int x) {
            if (x < 0) return -1;
            return x * x;
        }

    Decomposed steps:

    1. Validate the input `x` to ensure it is non-negative.
    2. If `x` is negative, return `-1` to indicate an error or invalid input.
    3. If `x` is non-negative, calculate the square of `x` (i.e., `x * x`).
    4. Return the result of the square calculation.

    ### Example 2:

    Original code:

        // Function that checks if a number is positive and returns a message
        string checkNumber(int x) {
            if (x > 0) return "Positive";
            return "Not Positive";
        }

    Decomposed steps:

    1. Check if the input number `x` is greater than zero.
    2. If `x` is positive, return the string "Positive".
    3. If `x` is not positive (i.e., zero or negative), return the string "Not Positive".

    Now, decompose the following function into logical steps based on its summary.
    Make it concise. No need to explain your reasoning or give examples:


    ''' + code19

In [ ]:
# Task 20
# Purpose Inference → Completion (Python) Complete the function based on intent:

code20 = '''
  def calculate_average ( scores ) :
    total = 0
    # TODO : Complete to return average
    pass

'''

few_shot_prompt20 = '''
    Your task is to complete the function based on its intent.

    ### Example 1:

    Original code:

        def calculate_average(scores):
            total = 0
            # TODO: Complete to return average
            pass

    Completed function:

        def calculate_average(scores):
            total = sum(scores)  # Sum all scores
            return total / len(scores) if scores else 0  # Return the average

    ### Example 2:

    Original code:

        def find_max(numbers):
            max_num = None
            # TODO: Complete to return the maximum number
            pass

    Completed function:

        def find_max(numbers):
            max_num = max(numbers)  # Use max function to get the maximum
            return max_num if numbers else None  # Return the maximum or None if the list is empty

    Now, complete the following function based on its intent.
    You only need to procide the completed function. no need for
    explanations or examples:

    Code
    ''' + code20

In [ ]:
# Task 21
# Full-File Bug Detection and Refactoring (Python) Analyze the following utility script. Identify any potential
# logic issues or design flaws. Then, refactor it for better readability, correctness, and safety.

code21 = '''
    # utils .py - Script to parse and summarize numeric CSV files
    import csv
    def read_csv ( filepath ) :
      with open ( filepath , ’r’) as f :
        return [ row for row in csv . reader ( f ) ]

    def summarize_column ( data , index ) :
      values = [ float ( row [ index ]) for row in data [1:]] # skip header
      total = sum( values )
      avg = total / len ( values )
      return total , avg

    def main () :
      filepath = ’data .csv ’
      data = read_csv ( filepath )
      total , avg = summarize_column ( data , 1)
      print (" Total :", total )
      print (" Average :", avg )

    if __name__ == ’__main__ ’:
      main ()

'''

few_shot_prompt21 = '''
    Your task is to analyze the utility script below for any potential logic issues, design flaws, or readability concerns. Then, refactor the code to improve correctness, safety, and readability.

    ### Example 1:

    Original code:

        import csv
        def read_csv(filepath):
            with open(filepath, 'r') as f:
                return [row for row in csv.reader(f)]

        def summarize_column(data, index):
            values = [float(row[index]) for row in data[1:]]  # skip header
            total = sum(values)
            avg = total / len(values)
            return total, avg

        def main():
            filepath = 'data.csv'
            data = read_csv(filepath)
            total, avg = summarize_column(data, 1)
            print("Total:", total)
            print("Average:", avg)

        if __name__ == '__main__':
            main()

    Refactored code:

        import csv

        def read_csv(filepath):
            """Reads a CSV file and returns a list of rows."""
            try:
                with open(filepath, 'r') as f:
                    return [row for row in csv.reader(f)]
            except FileNotFoundError:
                print(f"Error: The file at {filepath} does not exist.")
                return []

        def summarize_column(data, index):
            """Summarizes a column in the CSV data, calculating the total and average."""
            if not data:
                return 0, 0
            values = [float(row[index]) for row in data[1:]]  # skip header
            if len(values) == 0:
                return 0, 0
            total = sum(values)
            avg = total / len(values)
            return total, avg

        def main():
            """Main function to process CSV file and print summary."""
            filepath = 'data.csv'
            data = read_csv(filepath)
            if data:
                total, avg = summarize_column(data, 1)
                print("Total:", total)
                print("Average:", avg)
            else:
                print("No data to process.")

        if __name__ == '__main__':
            main()

    Explanation: The original script had a few issues:
    - If the file doesn't exist, the program would crash. We added exception handling for `FileNotFoundError`.
    - If the `data` list is empty or the column to summarize has no valid values, the program might divide by zero or fail. We added checks to return default values of `0` for both total and average in such cases.
    - Added docstrings to functions for better clarity.
    - Included checks to ensure the script gracefully handles errors and edge cases.

    ### Example 2:

    Original code:

        def calculate_area(radius):
            return 3.14 * radius ** 2

        def calculate_circumference(radius):
            return 2 * 3.14 * radius

        def main():
            r = float(input("Enter the radius: "))
            print("Area:", calculate_area(r))
            print("Circumference:", calculate_circumference(r))

    Refactored code:

        import math

        def calculate_area(radius):
            """Calculates area of a circle given radius."""
            return math.pi * radius ** 2

        def calculate_circumference(radius):
            """Calculates circumference of a circle given radius."""
            return 2 * math.pi * radius

        def main():
            """Main function to read input and calculate circle properties."""
            try:
                r = float(input("Enter the radius: "))
                print("Area:", calculate_area(r))
                print("Circumference:", calculate_circumference(r))
            except ValueError:
                print("Error: Please enter a valid numeric value for radius.")

        if __name__ == '__main__':
            main()

    Explanation: The refactoring here included:
    - Replaced the constant `3.14` with `math.pi` for more precision.
    - Added exception handling for non-numeric input for radius.
    - Included docstrings to explain the functions.

    Now, analyze and refactor the following script for improvements.
    You only need to provide the final refactored code. no need for
    explanation or examples:

    Code

    ''' + code21


In [ ]:
# Task 22
# Code Completion and Robustness Enhancement (Python) Complete the following file-processing script.
# The goal is to clean each line, remove punctuation, and count word frequencies correctly.

code22 = '''
# file_processor .py - Incomplete script for processing text files
import string

def load_file ( filepath ) :
  with open ( filepath , ’r’) as f :
    return f . readlines ()

def clean_line ( line ) :
  # TODO : Remove punctuation and make lowercase
  pass

def count_words ( lines ) :
  word_counts = {}
  for line in lines :
    clean = clean_line ( line )
  for word in clean . split () :
    word_counts [ word ] = word_counts . get ( word , 0) + 1
  return word_counts

def main () :
  filepath = ’input .txt ’
  lines = load_file ( filepath )
  counts = count_words ( lines )
  for word , count in sorted ( counts . items () ) :
    print ( f"{ word }: { count }")

if __name__ == ’__main__ ’:
  main ()

'''

few_shot_prompt22 = '''
    Your task is to complete and improve the following Python script that processes a text file. The script should clean each line by removing punctuation, converting the text to lowercase, and then count the frequency of each word in the file. Ensure robustness by handling edge cases.

    ### Example 1:

    Original script:

        import string

        def clean_line(line):
            # TODO: Remove punctuation and make lowercase
            pass

        def count_words(lines):
            word_counts = {}
            for line in lines:
                clean = clean_line(line)
            for word in clean.split():
                word_counts[word] = word_counts.get(word, 0) + 1
            return word_counts

        def main():
            filepath = 'input.txt'
            lines = load_file(filepath)
            counts = count_words(lines)
            for word, count in sorted(counts.items()):
                print(f"{word}: {count}")

        if __name__ == '__main__':
            main()

    Refactored code:

        import string

        def clean_line(line):
            """Removes punctuation, converts text to lowercase, and returns the cleaned line."""
            translator = str.maketrans('', '', string.punctuation)
            return line.translate(translator).lower()

        def count_words(lines):
            """Counts the frequency of each word in the provided lines."""
            word_counts = {}
            for line in lines:
                clean = clean_line(line)
                for word in clean.split():
                    word_counts[word] = word_counts.get(word, 0) + 1
            return word_counts

        def load_file(filepath):
            """Reads and returns lines from the given file."""
            with open(filepath, 'r') as f:
                return f.readlines()

        def main():
            """Main function to process the file and print word frequencies."""
            filepath = 'input.txt'
            lines = load_file(filepath)
            counts = count_words(lines)
            for word, count in sorted(counts.items()):
                print(f"{word}: {count}")

        if __name__ == '__main__':
            main()

    ### Example 2:

    Original script:

        import string

        def clean_line(line):
            # TODO: Remove punctuation and make lowercase
            pass

        def count_words(lines):
            word_counts = {}
            for line in lines:
                clean = clean_line(line)
            for word in clean.split():
                word_counts[word] = word_counts.get(word, 0) + 1
            return word_counts

        def main():
            filepath = 'input.txt'
            lines = load_file(filepath)
            counts = count_words(lines)
            for word, count in sorted(counts.items()):
                print(f"{word}: {count}")

        if __name__ == '__main__':
            main()

    Refactored code:

        import string

        def clean_line(line):
            """Removes punctuation and makes the text lowercase."""
            return line.translate(str.maketrans('', '', string.punctuation)).lower()

        def count_words(lines):
            """Counts the frequency of words from the provided lines."""
            word_counts = {}
            for line in lines:
                clean = clean_line(line)
                for word in clean.split():
                    word_counts[word] = word_counts.get(word, 0) + 1
            return word_counts

        def load_file(filepath):
            """Reads file and returns the lines."""
            with open(filepath, 'r') as f:
                return f.readlines()

        def main():
            """Processes the input file and prints word frequencies."""
            filepath = 'input.txt'
            lines = load_file(filepath)
            counts = count_words(lines)
            for word, count in sorted(counts.items()):
                print(f"{word}: {count}")

        if __name__ == '__main__':
            main()

    Now, analyze and complete the following script to
    process a file and count word frequencies. You
    only need to provide the final script. No need for
    explanations or examples:

    Code

    ''' + code22


In [ ]:
# for i in range(len(predictions)):
#   print("Task " + str(i+1))
#   print(predictions[i])
#   print()

Task 1
The method `countWordFrequency` takes a list of strings, counts the frequency of each word, and returns a map where the keys are the words and the values are their corresponding frequencies.

Task 2
The function `sum_range` is intended to calculate the sum of all integers from `start` to `end`, inclusive. However, the original function uses `range(start, end)`, which excludes the `end` value. This leads to an incomplete summation by omitting the final number.

Corrected Code:
```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```

Task 3
Returning pointer to local variable

Task 4
To complete the `is_valid_email` function using regex to validate basic email addresses, you can utilize a more detailed regex pattern that captures the typical structure of an email address. Here's the completed function:

```python
def is_valid_email(email):
    import re
    # Regex pattern to validate basic email addresses
    

In [ ]:
# All the prompts

few_shot_prompts = [
    few_shot_prompt1, few_shot_prompt2, few_shot_prompt3, few_shot_prompt4,
    few_shot_prompt5, few_shot_prompt6, few_shot_prompt7, few_shot_prompt8,
    few_shot_prompt9, few_shot_prompt10, few_shot_prompt11, few_shot_prompt12,
    few_shot_prompt13, few_shot_prompt14, few_shot_prompt15, few_shot_prompt16,
    few_shot_prompt17, few_shot_prompt18, few_shot_prompt19, few_shot_prompt20,
    few_shot_prompt21, few_shot_prompt22
]



We have defined all few-shot prompts. Now it's time to run them:



In [ ]:
# Store all few-shot prompts in a list
# Then stoe all the outputs in two seperate

GPT = []
Codestral = []
# Loop over each task
for i, few_shot_prompt in enumerate(few_shot_prompts, start=1):
    code_variable = globals().get(f'code{i}')  # Get the corresponding code dynamically

    for model in model_choices[:2]:  # Run on the first two models
        print(f"Running Task {i} on model: {model}")

        full_prompt = f"{few_shot_prompt}\nCode:\n{code_variable}\n"

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=1024,
            temperature=0.3,
        )

        output = response.choices[0].message.content

        print(f"Model: {model} | Task {i} Response:\n{response.choices[0].message.content}\n")

        if model == model_choices[0]:
            GPT.append(output)
        elif model == model_choices[1]:
            Codestral.append(output)

Running Task 1 on model: gpt-4o-mini
Model: gpt-4o-mini | Task 1 Response:
Summary:
Counts the frequency of each word in a list and returns a map with words as keys and their corresponding counts as values.

Running Task 1 on model: Codestral-2501
Model: Codestral-2501 | Task 1 Response:
The method `countWordFrequency` takes a list of strings and returns a map where each string is a key and its value is the frequency of that string in the list. It iterates through the list, updating the frequency count for each word in the map.

Running Task 2 on model: gpt-4o-mini
Model: gpt-4o-mini | Task 2 Response:
```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```

Running Task 2 on model: Codestral-2501
Model: Codestral-2501 | Task 2 Response:
The function `sum_range` is intended to calculate the sum of all integers from `start` to `end`, inclusive. However, the original code uses `range(start, end)`, which excludes the `

In [ ]:


# # Save as JSON files
# with open("GPT_Output_few_shot.json", "w") as f:
#     json.dump(GPT, f, indent=2)

# with open("Codestral_Output_few_shot.json", "w") as f:
#     json.dump(Codestral, f, indent=2)

In [ ]:
# GPT_outputs = []
# Codestral_outputs = []

# with open("GPT_Output_few_shot.json", "r") as f:
#     GPT_outputs = json.load(f)

# with open("Codestral_Output_few_shot.json", "r") as f:
#     Codestral_outputs = json.load(f)


In [ ]:
# Put them in seperate lists called references and predictions

references = GPT_outputs
predictions = Codestral_outputs

In [ ]:
# print(len(predictions), len(references))
# print(predictions)
# print(references)

22 22
['The method `countWordFrequency` takes a list of strings and returns a map where each string is a key and its value is the frequency of that string in the list. It iterates through the list, updating the frequency count for each word in the map.', 'The function `sum_range` is intended to calculate the sum of all integers from `start` to `end`, inclusive. However, the original code uses `range(start, end)`, which excludes the `end` value. This results in an incomplete summation by omitting the final number.\n\nCorrected Code:\n```python\ndef sum_range(start, end):\n    total = 0\n    for i in range(start, end + 1):\n        total += i\n    return total\n```', 'Returning pointer to local variable', "```python\ndef is_valid_email(email):\n    import re\n    pattern = r'^[a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$'\n    return re.match(pattern, email) is not None\n```", "```python\nfrom flask import Flask, jsonify\napp = Flask(__name__)\n\n@app.route('/greet/<username>')\ndef gr

In [ ]:
# Get the scores

Scores = []
selected_indices = {0, 2, 6, 9, 10, 18}

for i, pred, refs in zip(range(len(predictions)), predictions, references): # Use range to get the index
    # print("Task " + str(i+1))
    # print(pred)
    # print()
    # print("The next one: ")
    # print()
    # print(refs)
    # print()
    if i in selected_indices:
        result = sacrebleu.compute(predictions=[pred], references=[refs])
        Scores.append("Bleu-score: "+ str(round(result["score"], 5)))
    else:
      print()
      print(pred)
      print(refs)
      print()
      print("Exact Match: "+ str(pred == refs))
      Scores.append("Exact Match: "+ str(pred == refs))

print(Scores) # Print the calculated scores


The function `sum_range` is intended to calculate the sum of all integers from `start` to `end`, inclusive. However, the original code uses `range(start, end)`, which excludes the `end` value. This results in an incomplete summation by omitting the final number.

Corrected Code:
```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```
```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```

Exact Match: False

```python
def is_valid_email(email):
    import re
    pattern = r'^[a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None
```
```python
def is_valid_email(email):
    import re
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None
```

Exact Match: False

```python
from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/greet/<username>')
def greet(us

In [ ]:

# # Save as JSON files
# with open("Scores_Few_Shot.json", "w") as f:
#     json.dump(Scores, f, indent=2)

In [ ]:
# Scores_test = []

# with open("Scores_Few_Shot.json", "r") as f:
#     Scores_test = json.load(f)

In [ ]:
print(Scores_test)

['Bleu-score: 9.4884', 'Exact Match: False', 'Bleu-score: 54.88116', 'Exact Match: False', 'Exact Match: True', 'Exact Match: True', 'Bleu-score: 52.73931', 'Exact Match: False', 'Exact Match: True', 'Bleu-score: 82.74856', 'Bleu-score: 29.55261', 'Exact Match: True', 'Exact Match: False', 'Exact Match: False', 'Exact Match: False', 'Exact Match: False', 'Exact Match: True', 'Exact Match: False', 'Bleu-score: 100.0', 'Exact Match: True', 'Exact Match: False', 'Exact Match: False']


In [ ]:
print

In [ ]:
# analysis_text_GPT = ["Definately the simplest task for GPT to accomplish.\n It copied the examples we gave it one for one\n. The summay was succint and correct.",
#                      "Suprisingly, ChatGPT was the one to provide too much explanation this time. \nThe fix was correct, but it provided an explanation, \n as well as the fixed code twice. Very inefficient in interpreting the prompt.",
#                      "GPT followed the prompt examples basically perfectly. And the explanation seems correct. \n",
#                      "Both GPT and Codestral struggled with following the exact prompt examples. GPT \n provided more information than was asked for, providing an explanation and example usage. \n It was able to complete the regrex.",
#                      "Followed the prompt examples almost perfectly. It gave only the final code, which \n is what we were asking the model to return.",
#                      "Completes the schema. Worth noting that it used integer keys, which is straight from the prompt example \n It provides a short, non-intrusive explanation of each schema, which is helpful, but also was not asked for.",
#                      "Followed the prompt examples almost exactly. The explanation and suggested fix were succint.",
#                      "Fixed the parser to support quoted fields. Once again gave an explanation on what it did \n. This is odd, since in other examples, it is able to just provide the code. We're unsure \n what is different in our prompting that could be causing this.",
#                      ]

# analysis_text_codestral = ["As will be a theme with the rest of the Codestral, the model did not follow the prompt structure. \nIt gave a detailed explanation of the summary, which wasn't \n asked for and goes against the examples given. \Overall, a fairly huge failure.",
#                            "Much better than ChatGPT. While it still gave an explanation, it was only two lines. \n And the corrected code seems correct. It was much closer to examples.",
#                            "Codestral took the opposite approach to GPT. We don't know why, but it took \n the examples and the code and decided to \n produce its own examples of code with bug. \n This is one of the most extreme cases of adding fluff and not following the prompt examples.",
#                            "Codestral provided steps on what the completed code would need to do. \n This is helpful for learning. but it does not follow the prompt example. It does complete the code, but then also provides examples of it being used. \n It then explains the Regex pattern, which is completely added on by it."
#                            "The code itself is almost exactly the same as GPT. However, it once \n again provided as explanation as well as an example of the code being used. \n Once again, it does not follow the examples in the prompt.",
#                            "Very similar to GPT in the schema. However, the explanation \nis much more detailed, going into what the purpose of each line is.",
#                            "Did not follow the prompt examples. It gave the fixed code, then an explanation of what it did. \n It then provided additional considerations for the future, which are helpful. But again, not asked for.",
#                            "Provides an explanation of what needs to be fixed, then provides the code with \n example test cases. It then explains its characteristics."
#                            ]

In [ ]:
# for i in range(len(predictions)):
#   print("Task " + str(i+1))
#   print(predictions[i])
#   print()

Task 1
Summary:
Counts the frequency of each word in a list and returns a map with the words as keys and their corresponding counts as values.

Task 2
To correct the provided code, we need to ensure that the `sum_range` function properly includes both the `start` and `end` values in the summation. The issue arises from using `range(start, end)` which excludes the `end` value. The corrected version should use `range(start, end + 1)` to ensure that the `end` value is included in the sum.

Here’s the corrected code:

```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```

### Explanation of Changes:
- **Original Code**: `for i in range(start, end):` excludes the `end` value.
- **Corrected Code**: `for i in range(start, end + 1):` includes the `end` value in the summation by extending the range to `end + 1`. 

This adjustment ensures that the function sums all integers from `start` to `end`, inclusive.

Task 3
Bug Clas

In [ ]:
# Organize the output into a dataframe

data = []

for i, prompt in enumerate(few_shot_prompts):
    for model_name in ["Codestral-2501", "GPT-4o-mini"]:
      if model_name == "Codestral-2501":
        output = predictions[i]
      elif model_name == "GPT-4o-mini":
        output= references[i] # Your function
      score = Scores[i]  # Your metric
      analysis = "Generic text"  # Optional
      data.append({
            "prompt_id": i + 1,
            "parameters": "max_tokens=1024,\n temperature = 0.3",
            "prompt_text": prompt,
            "model_name": model_name,
            "output_text": output,
            "score": score,
            # "analysis": analysis
        })

df = pd.DataFrame(data)


In [ ]:
# Convert to csv

df.to_csv('few_shot_final.csv', index=False)